<a href="https://colab.research.google.com/github/Nimere1990/Multi-criteria-optimization/blob/main/%D0%95%D1%80%D1%91%D0%BC%D0%B8%D0%BD_%D0%98_%D0%95_%D0%94%D0%97_%E2%84%9647.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. Многокритериальная оптимизация**:
   1. Реализуйте многокритериальную оптимизацию гиперпараметров нейронной сети, учитывая не только точность, но и время обучения.
   2. Используйте метод весовых коэффициентов для объединения критериев.

In [ ]:
!pip install tensorflow
!pip install keras

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import time

In [ ]:
# 1. Загрузка и подготовка данных
wine_data = load_wine()  # Загружаем набор данных Wine
X = wine_data.data  # Извлекаем признаки (характеристики вин)
y = wine_data.target.reshape(-1, 1)  # Извлекаем целевую переменную (категории вин) и преобразуем в столбец

encoder = OneHotEncoder(sparse_output=False)  # Создаем объект кодировщика
y = encoder.fit_transform(y)  # Преобразуем целевую переменную в "горячий" код

# Разделяем данные на обучающую и тестовую выборки, оставляя 20% для тестирования
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# 2. Определение модели нейронной сети
def create_model(neurons, learning_rate):
    model = tf.keras.Sequential()  # Создаем последовательную модель нейронной сети
    # Добавляем скрытый слой с заданным количеством нейронов и функцией активации ReLU
    model.add(tf.keras.layers.Dense(neurons, input_dim=X_train.shape[1], activation='relu'))
    # Добавляем выходной слой для многоклассовой классификации с функцией активации softmax
    model.add(tf.keras.layers.Dense(y_train.shape[1], activation='softmax'))
    # Компилируем модель, используя категориальную кросс-энтропию в качестве функции потерь и Adam в качестве оптимизатора
    model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), metrics=['accuracy'])
    return model  # Возвращаем созданную модель

In [ ]:
# 3. Определение функции цели с учетом весов
def objective_function(neurons, learning_rate):
    start_time = time.time()  # Запоминаем время начала обучения

    model = create_model(neurons, learning_rate)  # Создаем нейронную сеть с заданными гиперпараметрами
    # Обучаем модель на обучающей выборке, скрывая вывод (verbose=0)
    model.fit(X_train, y_train, epochs=50, batch_size=5, verbose=0)

    # Получаем точность модели на тестовой выборке
    accuracy = model.evaluate(X_test, y_test, verbose=0)[1]

    elapsed_time = time.time() - start_time  # Вычисляем время, потраченное на обучение

    # Объединение критериев: 70% на точность и 30% на время
    combined_metric = 0.7 * accuracy - 0.3 * (elapsed_time / 20)  # Нормализуем время
    return combined_metric  # Возвращаем комбинированную метрику

In [ ]:
# 4. Реализация простого генетического алгоритма
def genetic_algorithm(pop_size, generations):
    # Инициализация популяции с произвольными значениями нейронов и скорости обучения
    population = [(np.random.randint(5, 50), np.random.uniform(0.001, 0.1)) for _ in range(pop_size)]

    best_individual = None  # Переменная для хранения лучших индивидуумов
    best_score = -np.inf  # Начальное значение для лучшей оценки

    # Основной цикл по поколениям
    for generation in range(generations):
        print(f"Поколение {generation + 1}/{generations}")  # Информируем о текущем поколении
        scores = []  # Список для хранения оценок каждого индивидуумa

        # Оценка индивидов в популяции
        for individual in population:
            neurons, learning_rate = individual  # Распаковываем параметры индивидуумa
            score = objective_function(neurons, learning_rate)  # Получаем комбинированную оценку
            scores.append(score)  # Сохраняем оценку индивидуумa

            # Проверяем, является ли текущая оценка лучшей
            if score > best_score:
                best_score = score  # Обновляем лучшую оценку
                best_individual = individual  # Сохраняем лучшие параметры

        # Создание новой популяции
        # Сортируем популяцию по оценкам и выбираем лучших
        ranked_population = [ind for _, ind in sorted(zip(scores, population), key=lambda x: x[0], reverse=True)]
        population = []  # Очистка списка для новой популяции

        # Выбор лучших индивидуумов для создания новой популяции
        for i in range(pop_size // 2):
            parent1 = ranked_population[i]  # Первый родитель
            parent2 = ranked_population[i + 1]  # Второй родитель

            # Кроссовер для создания новых индивидуумов
            child1_neurons = parent1[0]
            child2_neurons = parent2[0]
            if np.random.rand() < 0.5:  # Вероятность кроссовера
                child1_neurons = parent2[0]  # Меняем количество нейронов первого родителя
                child2_neurons = parent1[0]  # Меняем количество нейронов второго родителя

            # Мутация для изменения скорости обучения
            child1_learning_rate = parent1[1] + np.random.uniform(-0.01, 0.01)
            child2_learning_rate = parent2[1] + np.random.uniform(-0.01, 0.01)

            # Ограничиваем значения для детей
            child1 = (max(5, min(50, child1_neurons)), max(0.001, min(0.1, child1_learning_rate)))
            child2 = (max(5, min(50, child2_neurons)), max(0.001, min(0.1, child2_learning_rate)))
            population.extend([child1, child2])  # Добавляем детей в новую популяцию

    return best_individual, best_score  # Возвращаем лучшие параметры и лучшую оценку

In [ ]:
# 5. Оптимизация гиперпараметров
best_individual, best_score = genetic_algorithm(pop_size=10, generations=10)  # Запускаем генетический алгоритм

# Выводим лучшие гиперпараметры и их оценку
print("Лучшие гиперпараметры: Нейронов: {}, Скорость обучения: {:.4f}, Оценка: {:.4f}".format(int(best_individual[0]), best_individual[1], best_score))


Поколение 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Поколение 2/10
Поколение 3/10
Поколение 4/10
Поколение 5/10
Поколение 6/10
Поколение 7/10
Поколение 8/10
Поколение 9/10
Поколение 10/10
Лучшие гиперпараметры: Нейронов: 18, Скорость обучения: 0.0179, Оценка: 0.6411
